# Assessment Problems

Python project that downloads FAANG stock data, plots it, automates execution via a script, and schedules it weekly with GitHub Actions.

## Problem 1: Data from yFinance

Function `get_data()` uses [yfinance](https://github.com/ranaroussi/yfinance) Python package to:

1. Download hourly stock data for the past 5 days for FAANG (META, AAPL, AMZN, NFLX, GOOG).

2. Save results as a CSV in a data/ folder (create it if missing).

The filename format is `YYYYMMDD-HHmmss.csv` (date + time of save).

In [28]:
# Dates and times
import datetime as dt

# Data frames
import pandas as pd

#  Yahoo Finance yfinance module to fetch stock data
import yfinance as yf

Creating Tickers object for FAANG stocks
Tickers  
https://ranaroussi.github.io/yfinance#ticker


In [29]:
# Downloading stock data for Apple, Microsoft, Google, Amazon, and Tesla 
ticker_symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
# Fetching stock data using yfinance. It returns a Tickers dataframe
stock_data = yf.download(ticker_symbols, period="5d", interval="1h")

/var/folders/71/lgn75sbs3sngtkrhdkwngslm0000gn/T/ipykernel_71845/1717889272.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbols, period="5d", interval="1h")
[*********************100%***********************]  5 of 5 completed


Date and time formatting for filename  
https://docs.python.org/3/library/datetime.html

In [30]:
# Getting current date and time for filename with format YYYYMMDD-HHmmss
# https://docs.python.org/3/library/datetime.html#datetime.datetime.now
current_time = dt.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creating full file path (including the filename) for saving the stock data
full_filepath = f"data/{current_time}.csv"

Saving the fetched stock data to a CSV file  
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [31]:
# Saving the CSV file in data folder with name as per the specified format
stock_data.to_csv(full_filepath)
# Printing the fetched stock data
print(stock_data)

Price                           Close                                      \
Ticker                           AAPL        AMZN       GOOGL        MSFT   
Datetime                                                                    
2025-10-13 13:30:00+00:00  246.376007  220.110001  240.630005  515.070007   
2025-10-13 14:30:00+00:00  248.590302  220.160004  242.110001  512.840027   
2025-10-13 15:30:00+00:00  249.179993  220.169998  242.789993  513.790100   
2025-10-13 16:30:00+00:00  249.104202  220.050003  242.320099  513.210022   
2025-10-13 17:30:00+00:00  249.146194  220.524994  243.149994  514.150024   
2025-10-13 18:30:00+00:00  248.320007  220.119995  243.169998  514.030029   
2025-10-13 19:30:00+00:00  247.589996  220.000000  244.130005  513.969971   
2025-10-14 13:30:00+00:00  247.020004  216.425003  244.309998  510.690002   
2025-10-14 14:30:00+00:00  246.214005  216.494995  245.225006  510.678406   
2025-10-14 15:30:00+00:00  247.389999  218.300003  246.559998  512.602478   

## Problem 2: Plotting Data

Function `plot_data()` loads the latest CSV in the `data` folder, plots `Close` prices for all five FAANG stocks with labels, legend, and date as the title.
Then it saves the plot in a `plots` folder as `YYYYMMDD-HHmmss.png` by creating the folder if it doesn't exist.

## Problem 3: Script

Script `faang.py` in the repo root contains the two functions (`get_data()` and `plot_data()`). It can be run by `./faang.py` from the terminal to download the data and generate the plot.

## Problem 4: Automation

GitHub Actions workflow (`faang.yml`) inside `.github/workflows/` runs `faang.py` every Saturday morning.

## End